In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import copy


import openpyxl
cf.go_offline(connected= True)

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
lst_a = [1,-1,3,-1]

for idx,i in enumerate(lst_a):
    if type(i) == int:
        if i < 0:
            lst_a[idx] = 0
            
lst_a

[1, 0, 3, 0]

In [48]:
df_A.drop(columns = [i for i in df_A.columns[1:3]])
df_A = 

,0,3,4,5,6,7,8,9,10,11,12
Forecast,0,100,120,150,150,150,200,200,200,200,200
Order,0,50,30,20,10,0,0,0,0,0,0
PAB,160,0,0,0,0,0,0,0,0,0,0
ATP,0,0,0,0,0,0,0,0,0,0,0
MPS_receipts,0,0,0,0,0,0,0,0,0,0,0
MPS,0,0,0,0,0,0,0,0,0,0,0


In [128]:
class Production:
    
    def __init__(self, df, bom_graph, bom_spec ,start_node,  mode = 'Chasing',lead_time =0 , num = 0, already = 0):
        self.df = df
        self.mode = mode
        self.bom_graph = bom_graph
        self.bom_spec = bom_spec
        self.start_node = start_node
        self.mode = mode
        self.lead_time = lead_time
        self.num = num
        self.start_node = start_node 
        self.prior_order = []
        self.prior_forecast = []
        self.mps_ori = []
        
        self.already = already
        
        self.df.columns = [i for i in range(df.shape[1])]
        
    # MPS :주일정계획(master production schedule: MPS): 주(週) 또는 일(日)별로, 생산되어야 할 구체적인 제품 또는 제품군의 양을 명시한 것. 을 함수화
        
    def product_ATP(self,df):
        
        df = df.copy()

        ATP_lst = []
        ATP_sum = []
        idx = 0
        idx_standard = 0

        for idx in range(df.shape[1]):

            if idx == 0:
                pass

            elif idx ==1 :

                num = 0

                while 1:


                    if idx >= (df.shape[1]):
                        break

                    if df.loc['MPS_receipts',idx] != 0:

                        if  num == 0:
                            ATP_lst.append(df.loc['MPS_receipts',idx_standard] - df.loc['Order',idx_standard])
                            ATP_sum = []
                            break

                        ATP_lst.append(df.loc['PAB',idx_standard-1] - sum([int(i) for i in ATP_sum]))

                        ATP_sum = []                  
                        break

                    elif df.loc['MPS_receipts',idx] == 0:
                        ATP_sum.append(df.loc['Order',idx])

                    else:
                        pass

                    num += 1
                    idx += 1

            else:
                num = 0

                while 1:

                    if idx >= (df.shape[1]):
                        break

                    if df.loc['MPS_receipts',idx] != 0:

                        if  num == 0:
                            ATP_lst.append(df.loc['MPS_receipts',idx_standard] - df.loc['Order',idx_standard])

                            ATP_sum = []
                            break

                        a = df.loc['MPS_receipts',idx_standard] - sum(ATP_sum)

                        if a <= 0:
                            ATP_lst.append(0)
                            break


                        else:
                            ATP_lst.append( df.loc['MPS_receipts',idx_standard] - sum(ATP_sum))

                            ATP_sum = []

                            break

                    elif df.loc['MPS_receipts',idx] == 0:
                        

                        ATP_sum.append(df.loc['Order',idx])
                        ATP_lst.append('No MPS, No ATP')
                        break

                    else:
                        pass

                    idx += 1                    
                    num += 1                

            idx_standard += 1

            
            
        for idx,i in enumerate(ATP_lst):
            try:
                if i < 0:
                    ATP_lst[idx] = 0
            except:
                pass
                
        

        df.loc['ATP',1:] = ATP_lst

        return df

    
    def prior(self,df):
        df = df.copy()
        prior_order = self.prior_order
        prior_forecast = self.prior_forecast  
        mps_ori = self.mps_ori
        
        already = self.already
        
        lead_time = self.bom_spec[self.start_node][2]
        num =  self.bom_spec[self.start_node][1]
        
        for idx,i,j in zip(df.columns, df.loc['Forecast',:], df.loc['MPS',:]):
            
            if j > 0:
                if df.loc['Forecast',idx + lead_time ] >= df.loc['Order',idx + lead_time ]:
                    ww = df.loc['Forecast',idx + lead_time ]
                    df.loc['Forecast',idx + lead_time ] -= j
                    self.prior_forecast.append((idx + lead_time -already , ww))
                    
                    if df.loc['Forecast',idx + lead_time ]>= df.loc['Order',idx + lead_time ]:
                        pass
                    else:
                        ww_2 = df.loc['Order',idx + lead_time ]
                        df.loc['Order',idx + lead_time ] =  df.loc['Forecast',idx + lead_time ]
                        self.prior_order.append((idx + lead_time -already , ww_2))
                        
                        
                else:
                    ww_3 = df.loc['Order',idx + lead_time ]
                    df.loc['Order',idx + lead_time ] -= j
                    self.prior_order.append((idx + lead_time -already , ww_3))
                    
                self.mps_ori.append((idx + lead_time -already , j))
                df.loc['MPS',idx] = 0
            
            else:

                   pass
                       
                
        if already != 0:
            df.loc['PAB',0] = df.loc['PAB',0] - sum(df.loc['Order',1:already])  
            df = df.drop(columns = [i for i in df.columns[1:already+1]])                  
            df.columns = [i for i in range(df.shape[1])]        
      
            
        return df
            
            
            
    def after(self,df):
        
        df = df.copy()
        prior_order = self.prior_order
        prior_forecast = self.prior_forecast  
        mps_ori = self.mps_ori
        already = self.already
        
        for idx,value in prior_order:
            df.loc['Order' ,idx] = value
            
        for idx,value in prior_forecast:
            df.loc['Forecast', idx] = value
            
        for idx,value in mps_ori:
            df.loc['MPS_receipts', idx] += value
            
            

            
        print(df)
        return df
        
        
        
        
    
    
    def product_MPS(self, df):
        
        df = df.copy()
        
        lead_time = self.bom_spec[self.start_node][2] 
        num = self.bom_spec[self.start_node][2] 
        
        MPS_lst = []
        for idx, scheduled in enumerate(df.loc['MPS',1:]):

            if  idx == 0:
                idx = idx+1
                w = -df.loc['PAB',idx+lead_time-1] + max(df.loc['Forecast',idx+lead_time],df.loc['Order',idx+lead_time])
                if w < 0:
                    w = 0
                MPS_lst.append(w)
                                                                      
            else:
                try:
                    idx = idx+1
                    w_2 = -df.loc['PAB',idx+lead_time-1] + max(df.loc['Forecast',idx+lead_time],df.loc['Order',idx+lead_time])
                    if w_2 <0:
                        w_2 = 0
                    MPS_lst.append(w_2)
                except:
                    MPS_lst.append(0)



        df.loc['MPS',1:] = MPS_lst

        return df
    

    
    def product_MPS_receipts(self, df ):
        df = df.copy()
        
        lead_time = self.bom_spec[self.start_node][2]
        num =  self.bom_spec[self.start_node][1]

        MPS_receipts_lst = [0] * df.shape[1] 
        MPS_lst = df.loc['MPS',:].tolist()


        plus = [0] * lead_time
        MPS_receipts_lst = plus + MPS_lst 

        MPS_receipts_lst = MPS_receipts_lst[:-lead_time] 

        df.loc['MPS_receipts',:] = MPS_receipts_lst

        return df 
  


    def product_PAB(self, df ):
        df = df.copy()


        PAB_lst = []

        for idx,j in enumerate(zip(df.loc['Forecast',:],df.loc['Order',:],df.loc['MPS_receipts',:])):

            if idx == 0:
                PAB_lst.append(df.loc['PAB',idx])

                
            else:
                PAB_lst.append(PAB_lst[idx - 1] - max(int(j[0]),int(j[1])) + int(j[2]))

        df.loc['PAB',:] = PAB_lst

        return df
    
    # MPS 최종 데이터 적용.
    def MPS_apply(self,df):
        
        df = self.df.copy()
        df = self.prior(df)

        for _ in range(df.shape[1]):
                 
            df = self.product_MPS_receipts(self.product_MPS(self.product_PAB(df)))

        df = self.after(df)
        
        df = self.product_ATP(df)
        
        
        return df
    
    # MRP 만들기
    def MRP_make(self, df, lead_time , num , mode = 'chasing'):
        
        df = df.copy()

        part = {'Gross_requirements' : [0]*df.shape[1] , 'scheduled_receipts' : [0]*df.shape[1], 'Projected_Inventory_Balanance' : [0] * df.shape[1] , 
            'Net_requirements' : [0]*df.shape[1] , 'Planned_receipts' : [0]*df.shape[1], 'Planned_order_release' : [0]*df.shape[1]} 

        # MPS 처음 만들시 (MPS로부터 MRP를 처음 만들시)
        if 'Forecast' in df.index:
                
            df_new = pd.DataFrame(part).T
            df_new.loc['Gross_requirements',:]  = df.loc['MPS',:] * num
            df_new.loc['Planned_receipts'] = df_new.loc['Gross_requirements',:]

            new_lst = []
            new_lst = df_new.loc['Planned_receipts',:].tolist() + [0]*lead_time

            df_new.loc['Planned_order_release',:] = new_lst[lead_time:]
     
            
            
            
            df_new.loc['Projected_Inventory_Balanance',0] = df_new.loc['Gross_requirements',0] + df_new.loc['Gross_requirements',1]
            df_new.loc['Planned_order_release',0] = 0 
            
            
            
            for idx,i in enumerate(df_new.loc['Gross_requirements',:]):
                
                idx += 1
                
                if idx == 1:
                    df_new.loc['Planned_receipts',1] = 0

                
                elif (idx-lead_time) < 0:
                    df_new.loc['Planned_receipts',idx] = 0
                    df_new.loc['scheduled_receipts',idx] = df_new.loc['Gross_requirements',idx] 
            
                elif idx == lead_time:
                    df_new.loc['Planned_receipts',idx] = 0
                    df_new.loc['scheduled_receipts',idx] = df_new.loc['Gross_requirements',idx]
            
                else:
                    pass
                
            
        # MRP로부터 MRP 만들시
        else:

            df_new = pd.DataFrame(part).T
            df_new.loc['Gross_requirements',:]  = df.loc['Planned_order_release',:] * num
            df_new.loc['Planned_receipts'] = df_new.loc['Gross_requirements',:]

            new_lst = []
            new_lst = df_new.loc['Planned_receipts',:].tolist() + [0]*lead_time

            df_new.loc['Planned_order_release',:] = new_lst[lead_time:]
            

            
            df_new.loc['Projected_Inventory_Balanance',0] = df_new.loc['Gross_requirements',0] + df_new.loc['Gross_requirements',1]
            df_new.loc['Planned_order_release',0] = 0 
            
            for idx,i in enumerate(df_new.loc['Gross_requirements',:]):
                
                idx += 1
                
                if idx == 1:
                    df_new.loc['Planned_receipts',idx] = 0

                
                elif (idx-lead_time) < 0:
                    df_new.loc['Planned_receipts',idx] = 0
                    df_new.loc['scheduled_receipts',idx] = df_new.loc['Gross_requirements',idx] 
                    
                elif idx == lead_time:
                    df_new.loc['Planned_receipts',idx] = 0
                    df_new.loc['scheduled_receipts',idx] = df_new.loc['Gross_requirements',idx]

                
                else:
                    pass


        lst = []    
        # MRP, MPS's column, row -> index represent.
        for i,j,k in zip(df_new.iloc[2,:-1] ,df_new.iloc[1,1:] ,df_new.iloc[0,1:]): # Projected_Inventory_Balanance , scheduled_receipts , Gross_requirements
            lst.append(abs(i+j-k))
            
            
        lst.insert(abs(df_new.iloc[2,-2] + df_new.iloc[1,-1] - df_new.iloc[0,-1]),0)
        
        # Net requirements -> assignments.
        df_new.iloc[3,:] = lst
            
                
        return df_new
    
    
    # bom 에서 lead time과 num만 뽑아 낼 수 있으면 된다.
    def dfs(self, graph, start_node):

        need_visited, visited = list(), list()
        need_visited.append(start_node)

        while need_visited:
            node = need_visited.pop()

            if node not in visited:
                visited.append(node)
                need_visited.extend(graph[node])

        return visited

        
            
    # MPS, MRP 실행
    def make_MRP_MPS(self,file_name):
        
        df_new = self.df.copy()
        bom_graph = self.bom_graph.copy()
        bom_spec = self.bom_spec.copy()
        start_node = self.start_node
        
        df_MPS = self.MPS_apply(df_new).copy()

        
        ww = df_MPS.shape[1]            
        writer = pd.ExcelWriter(str(file_name)+'.xlsx', engine='xlsxwriter')
        
        
        
        lst_dict = {}
        lst_level = []
        
        for i in self.dfs(bom_graph, start_node):

            level  = bom_spec[i][0]
            num = bom_spec[i][1]
            lead_time = bom_spec[i][2]

            
            if level == 1:                
                
                df_f = df_MPS.copy()
                
                if ww == 13:
                    df_f.columns = [i for i in range(13)]
                    pass
                else :
                    df_f.columns = [0] + [i + (14 -ww ) for i in range(ww-1)]
                    
                
                df_f.to_excel(writer, sheet_name= 'level_'+str(level)+'_'+ i)
        

                    
                
                lst_level.append(level)
                df_d = df_MPS.copy()
                lst_dict[level] = df_d

            
            elif level == 2:
                df_d = self.MRP_make(df_MPS, lead_time =lead_time, num=num)
                
                df_f = df_d.copy()
                
                if ww == 13:
                    df_f.columns = [i for i in range(13)]
                    pass
                else :
                    df_f.columns = [0] + [i + (14 -ww ) for i in range(ww-1)] 
                
                
                df_f.to_excel(writer, sheet_name= 'level_'+str(level)+'_'+ i)
                lst_level.append(level)
                lst_dict[level] = df_d
                
            else:
                
                if level < lst_level[-1]:
                    df_d = lst_dict[level-1]
                    df_c = self.MRP_make(df_d, lead_time =lead_time, num=num)
                    

                    

                    lst_dict[level] = df_c

                
                elif level == lst_level[-1]:
                    df_d = lst_dict[level-1]
                    df_c = self.MRP_make(df_d, lead_time =lead_time, num=num)
          
                else:
                    df_d = lst_dict[level-1]
                    df_c = self.MRP_make(df_d, lead_time =lead_time, num=num)
                    


                    lst_dict[level] = df_c
                    
                df_f = df_c.copy()


                if ww == 13:
                    df_f.columns = [i for i in range(13)]
                    pass
                else :
                    df_f.columns = [0] + [i + (14 -ww ) for i in range(ww-1)]
                    
                    
                df_f.to_excel(writer, sheet_name= 'level_'+str(level)+'_'+ i)                    
                lst_level.append(level)

        writer.save()
        writer.close()
        
        if ww == 13:
            df_MPS.columns = [i for i in range(13)]
            pass
        else :
            df_MPS.columns = [0] + [i + (14 -ww ) for i in range(ww-1)]   
            

        return df_MPS


# 넣는 순서
- 가장 깊은 곳까지 가는 Node를 맨 뒤로
- 위의 노드는 가장 앞으로
- 같은 깊이라면 순서 상관 없으나 둘 다 가장 뒤에 넣는 것은 상관없음


# 주의사항

- 만약 pd.read_csv 시 오류가 나면, column명들이 str로 되어서 그럴 수 있다. 이때는 컬럼명들을 int 형으로 바꿀 것! ([int(i) for i in df.columns)]

In [135]:
# Bill of Material(BOM) 작성

# 제품 또는 Component가 어떤 Component와 연결되어 있는지를 보여줌
bom_dict = {} 
bom_dict['A'] = ['D']
bom_dict['D'] = ['A','E','F','G']
bom_dict['E'] = ['D']
bom_dict['F'] = ['D']
bom_dict['G']= ['D']

# level ,num , LT 순으로 씀
bom_dict_2 = {} 
bom_dict_2['A'] = [1,1,2]
bom_dict_2['D'] = [2,1,1]
bom_dict_2['E'] = [3,2,1]
bom_dict_2['F'] = [3,3,2]
bom_dict_2['G'] = [3,1,1]

In [137]:
# Product 지정,  BOM_1 (연결) , BOM_2 (LT,num,level) , 시작 노드 (Product 이름)  Input
df_A = pd.read_excel('test_A.xlsx',index_col= 0)
df_A.columns = [int(i) for i in df_A.columns ]

A = Production(df_A,bom_graph= bom_dict, bom_spec= bom_dict_2 ,start_node= 'A',already=2) 
A.make_MRP_MPS('make_A_test')

              0    1    2    3    4    5    6    7    8    9    10
Forecast       0  100  120  150  150  150  200  200  200  200  200
Order          0   50   30   20   10    0    0    0    0    0    0
PAB          -80  -20  -40    0    0    0    0    0    0    0    0
ATP            0    0    0    0    0    0    0    0    0    0    0
MPS_receipts   0  160  100  190  150  150  200  200  200  200  200
MPS            0  190  150  150  200  200  200  200  200    0    0


,0,3,4,5,6,7,8,9,10,11,12
Forecast,0,100,120,150,150,150,200,200,200,200,200
Order,0,50,30,20,10,0,0,0,0,0,0
PAB,-80,-20,-40,0,0,0,0,0,0,0,0
ATP,0,110,70,170,140,150,200,200,200,200,200
MPS_receipts,0,160,100,190,150,150,200,200,200,200,200
MPS,0,190,150,150,200,200,200,200,200,0,0


In [108]:
# Product 지정,  BOM_1 (연결) , BOM_2 (LT,num,level) , 시작 노드 (Product 이름)  Input
# 초기 1주차가 Actual value가 150, 원래부터 발주한 MPS가 160으로 설정한다. 
df_A = pd.read_excel('test_A_altered.xlsx',index_col= 0)
df_A.columns = [int(i) for i in df_A.columns ]

A = Production(df_A,bom_graph= bom_dict, bom_spec= bom_dict_2 ,start_node= 'A',already=1) 
A.make_MRP_MPS('make_A_altered')

              0    1    2    3    4    5    6    7    8    9    10   11
Forecast       0  100  100  120  150  150  150  200  200  200  200  200
Order          0   80   50   30   20   10    0    0    0    0    0    0
PAB           10  -90  -30    0    0    0    0    0    0    0    0    0
ATP            0    0    0    0    0    0    0    0    0    0    0    0
MPS_receipts   0    0  160  150  150  150  150  200  200  200  200  200
MPS            0  150  150  150  150  200  200  200  200  200    0    0


,0,2,3,4,5,6,7,8,9,10,11,12
Forecast,0,100,100,120,150,150,150,200,200,200,200,200
Order,0,80,50,30,20,10,0,0,0,0,0,0
PAB,10,-90,-30,0,0,0,0,0,0,0,0,0
ATP,0,0,110,120,130,140,150,200,200,200,200,200
MPS_receipts,0,0,160,150,150,150,150,200,200,200,200,200
MPS,0,150,150,150,150,200,200,200,200,200,0,0


# 무조건 깊은 노드를 뒤에!

In [99]:
bom_dict = {}
bom_dict['B'] = ['E','H']
bom_dict['E'] = ['B']
bom_dict['H'] = ['B','F','I']
bom_dict['F'] = ['H']
bom_dict['I'] = ['H']


bom_dict_2 = {} # level, num , LT 순으로 씀
bom_dict_2['B'] = [1,1,1]
bom_dict_2['E'] = [2,2,1]
bom_dict_2['H'] = [2,1,2]
bom_dict_2['F'] = [3,2,2]
bom_dict_2['I'] = [3,3,1]



In [100]:
df_B = pd.read_excel('test_B.xlsx',index_col= 0)
df_B.columns = [int(i) for i in df_B.columns ]

B = Production(df_B,bom_graph= bom_dict, bom_spec= bom_dict_2 ,start_node= 'B') 
B.make_MRP_MPS('make_B')

               0    1    2    3    4    5    6    7    8    9    10   11   12
Forecast        0  150  150  180  180  180  180  230  230  230  230  230  230
Order           0  180  120   80   50   30   20    0    0    0    0    0    0
PAB           200   20    0    0    0    0    0    0    0    0    0    0    0
ATP             0    0    0    0    0    0    0    0    0    0    0    0    0
MPS_receipts    0    0  130  180  180  180  180  230  230  230  230  230  230
MPS             0  130  180  180  180  180  230  230  230  230  230  230    0


,0,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,0,150,150,180,180,180,180,230,230,230,230,230,230
Order,0,180,120,80,50,30,20,0,0,0,0,0,0
PAB,200,20,0,0,0,0,0,0,0,0,0,0,0
ATP,0,20,10,100,130,150,160,230,230,230,230,230,230
MPS_receipts,0,0,130,180,180,180,180,230,230,230,230,230,230
MPS,0,130,180,180,180,180,230,230,230,230,230,230,0


In [120]:

# 초기 1주차가 Actual value가 150, 원래부터 발주한 MPS가 130으로 설정한다. 
df_B = pd.read_excel('test_B_altered.xlsx',index_col= 0)
df_B.columns = [int(i) for i in df_B.columns ]

B = Production(df_B,bom_graph= bom_dict, bom_spec= bom_dict_2 ,start_node= 'B',already= 1) 
B.make_MRP_MPS('make_B_altered')

               0    1    2    3    4    5    6    7    8    9    10   11
Forecast        0  150  180  180  180  180  230  230  230  230  230  230
Order           0  120   80   50   30   20    0    0    0    0    0    0
PAB           200  180    0    0    0    0    0    0    0    0    0    0
ATP             0    0    0    0    0    0    0    0    0    0    0    0
MPS_receipts    0  130    0  180  180  180  230  230  230  230  230  230
MPS             0    0  180  180  180  230  230  230  230  230  230    0


,0,2,3,4,5,6,7,8,9,10,11,12
Forecast,0,150,180,180,180,180,230,230,230,230,230,230
Order,0,120,80,50,30,20,0,0,0,0,0,0
PAB,200,180,0,0,0,0,0,0,0,0,0,0
ATP,0,10,"No MPS, No ATP",130,150,160,230,230,230,230,230,230
MPS_receipts,0,130,0,180,180,180,230,230,230,230,230,230
MPS,0,0,180,180,180,230,230,230,230,230,230,0


In [101]:
bom_dict = {}
bom_dict['C'] = ['G','J','H']
bom_dict['G'] = ['C']
bom_dict['J'] = ['C']
bom_dict['H'] = ['C','F','I']
bom_dict['F'] = ['H']
bom_dict['I'] = ['H']

bom_dict_2 = {} # level, num , LT 순으로 씀
bom_dict_2['C'] = [1,1,1]
bom_dict_2['G'] = [2,2,1]
bom_dict_2['J'] = [2,1,1]
bom_dict_2['H'] = [2,2,2]
bom_dict_2['F'] = [3,2,2]
bom_dict_2['I'] = [3,3,1]

In [102]:

df_C = pd.read_excel('test_C.xlsx',index_col= 0)
df_C.columns = [int(i) for i in df_B.columns ]

C = Production(df_C,bom_graph= bom_dict, bom_spec= bom_dict_2 ,start_node= 'C') 
C.make_MRP_MPS('make_C')

               0    1    2    3    4    5    6    7    8    9    10   11   12
Forecast        0  200  200  200  240  240  240  280  280  280  280  280  280
Order           0  200  180  120   80   40   10    0    0    0    0    0    0
PAB           240   40    0    0    0    0    0    0    0    0    0    0    0
ATP             0    0    0    0    0    0    0    0    0    0    0    0    0
MPS_receipts    0    0  160  200  240  240  240  280  280  280  280  280  280
MPS             0  160  200  240  240  240  280  280  280  280  280  280    0


,0,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,0,200,200,200,240,240,240,280,280,280,280,280,280
Order,0,200,180,120,80,40,10,0,0,0,0,0,0
PAB,240,40,0,0,0,0,0,0,0,0,0,0,0
ATP,0,40,0,80,160,200,230,280,280,280,280,280,280
MPS_receipts,0,0,160,200,240,240,240,280,280,280,280,280,280
MPS,0,160,200,240,240,240,280,280,280,280,280,280,0


In [123]:
# 초기 1주차가 Actual value가 150, 원래부터 발주한 MPS가 160으로 설정한다. 
df_C = pd.read_excel('test_C_altered.xlsx',index_col= 0)
df_C.columns = [int(i) for i in df_C.columns ]

C = Production(df_C,bom_graph= bom_dict, bom_spec= bom_dict_2 ,start_node= 'C',already= 1) 
C.make_MRP_MPS('make_C_altered')

               0    1    2    3    4    5    6    7    8    9    10   11
Forecast        0  200  200  240  240  240  280  280  280  280  280  280
Order           0  180  120   80   40   10    0    0    0    0    0    0
PAB           240  200    0    0    0    0    0    0    0    0    0    0
ATP             0    0    0    0    0    0    0    0    0    0    0    0
MPS_receipts    0  160    0  240  240  240  280  280  280  280  280  280
MPS             0    0  240  240  240  280  280  280  280  280  280    0


,0,2,3,4,5,6,7,8,9,10,11,12
Forecast,0,200,200,240,240,240,280,280,280,280,280,280
Order,0,180,120,80,40,10,0,0,0,0,0,0
PAB,240,200,0,0,0,0,0,0,0,0,0,0
ATP,0,0,"No MPS, No ATP",160,200,230,280,280,280,280,280,280
MPS_receipts,0,160,0,240,240,240,280,280,280,280,280,280
MPS,0,0,240,240,240,280,280,280,280,280,280,0


In [103]:
import openpyxl

class Final:
    
    def __init__(self,Total_name,filenames):
        self.filenames = filenames
        self.Total_name = Total_name
    
    def aggregation(self):
        filenames =  self.filenames
        lst_all  = []
        for file in filenames:
            wb = openpyxl.load_workbook(str(file) + '.xlsx')
            ws_names = wb.sheetnames
            for i in ws_names:
                lst_all.append((i[-1],pd.read_excel(str(file)+'.xlsx',sheet_name = i,index_col=0)))

        return lst_all


    def summation(self,lst):
        
        word = []
        for i in lst:
            word.append(i[0])

        word = list(set(word))

        writer = pd.ExcelWriter(str(self.Total_name)+'.xlsx', engine='xlsxwriter')
        for k in word:
            a = [i[1] for i in lst if i[0] == k ]

            df_0 = a[0]

            if len(a) == 1:
                pass

            else:
                for df in a[1:]:
                    df_0 += df


            df_0.to_excel(writer, sheet_name= str(k))

        writer.save()
        writer.close()          

        return df_0

    
    def agg_sum(self):
        
        A = self.summation(self.aggregation())
        
        return A
        


In [104]:
filenames = ['make_A','make_B','make_C']

make_ingredient = Final('ALL',filenames)
make_ingredient.agg_sum()
       

,0,1,2,3,4,5,6,7,8,9,10,11,12
Gross_requirements,0,450,580,660,660,660,790,790,790,790,790,790,0
scheduled_receipts,0,0,580,0,0,0,0,0,0,0,0,0,0
Projected_Inventory_Balanance,450,0,0,0,0,0,0,0,0,0,0,0,0
Net_requirements,0,0,0,660,660,660,790,790,790,790,790,790,0
Planned_receipts,0,0,0,660,660,660,790,790,790,790,790,790,0
Planned_order_release,0,660,660,660,790,790,790,790,790,790,0,0,0


In [280]:
def aggregation(filenames):

    lst_all  = []
    for file in filenames:
        wb = openpyxl.load_workbook(str(file) + '.xlsx')
        ws_names = wb.sheetnames
        for i in ws_names:
            lst_all.append((i[-1],pd.read_excel(str(file)+'.xlsx',sheet_name = i,index_col=0)))

    return lst_all

In [281]:
def summation(lst):
    word = []
    for i in lst:
        word.append(i[0])

    word = list(set(word))

    writer = pd.ExcelWriter(str('ALL')+'.xlsx', engine='xlsxwriter')
    for k in word:
        a = [i[1] for i in lst if i[0] == k ]

        df_0 = a[0]
        print(df_0)
        if len(a) == 1:
            pass
            
        else:
            for df in a[1:]:
                df_0 += df
        

        df_0.to_excel(writer, sheet_name= str(k))

    writer.save()
    writer.close()          

    return a
        
